# Phase 4 Project: Time Series Modeling

##  Forecasting Real Estate Prices Using Time Series

* Student Name: Andrea Cabello

## I. Overview 

I have been hired by a real-estate investment firm to help them understand:
- How real estate prices have changed overtime and, 
- How to use this information to invest in areas with the most potential value increase.

To get me started I have been provided with a dataset from [Zillow Research](https://www.zillow.com/research/data/)

### Home values

Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing type. It reflects the typical value for homes in the 35th to 65th percentile range. The raw raw version of that mid-tier ZHVI time series is also available.


* The firm is particularly interested in the North East Coast so I will focus my analysis on NY, NJ, CT and PA zipcodes.

## II. Business Problem

* What are the top 5 best zip codes for us to invest in?

Let's begin by defining best: 
* What makes the value of a property go up?
* Law of Supply and Demand. e movements. Property values rise when a low supply of homes for sale meets strong buyer demand, as buyers compete in bidding wars to secure a home from the limited inventory.

* Return On Investment
* Average price over time
* Jobs Market
* Population growth
* Access to the city center or other relevant landmarks

In [86]:
import pandas as pd

In [87]:
df = pd.read_csv('zillow_data.csv')

In [88]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [89]:
states = ['NY', 'CT', 'NJ','PA']
df = df[df.State.isin(states)].reset_index()
df.drop(['index', 'RegionID'], axis=1, inplace=True)
df1 = df.drop(columns=['City','State','Metro','CountyName', 'SizeRank'], axis=1)

In [90]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2472 entries, 0 to 2471
Columns: 271 entries, RegionName to 2018-04
dtypes: float64(219), int64(48), object(4)
memory usage: 5.1+ MB


,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,10467,New York,NY,New York,Bronx,7,152900.0,152700.0,152600.0,152400.0,...,394400,400000,407300,411600,413200,414300,413900,411400,413200,417900
1,11226,New York,NY,New York,Kings,11,162000.0,162300.0,162600.0,163000.0,...,860200,851000,853900,870000,885100,887800,890500,901700,930700,963200
2,11375,New York,NY,New York,Queens,13,252400.0,251800.0,251400.0,251200.0,...,1022600,1033700,1048600,1066400,1081200,1088800,1092700,1089500,1084000,1084600
3,11235,New York,NY,New York,Kings,14,190500.0,191000.0,191500.0,192000.0,...,767300,777300,788800,793900,796000,799700,806600,810600,813400,816200
4,10011,New York,NY,New York,New York,21,NaN,NaN,NaN,NaN,...,12137600,12112600,12036600,12050100,12016300,11946500,11978100,11849300,11563000,11478300


In [91]:
print(f'Number of zipcodes: {len(df)}')

Number of zipcodes: 2472


# Step 2: Data Preprocessing

In [92]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [93]:
datetimes = get_datetimes(df1)

In [94]:
datetimes

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

# Step 3: EDA and Visualization

In [95]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

NameError: name 'matplotlib' is not defined

# Step 4: Reshape from Wide to Long Format

In [105]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], 
                     var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [106]:
melted_df = melt_data(df)

In [107]:
melted_df.head()

,RegionName,City,State,Metro,CountyName,SizeRank,time,value
0,10467,New York,NY,New York,Bronx,7,1996-04-01,152900.0
1,11226,New York,NY,New York,Kings,11,1996-04-01,162000.0
2,11375,New York,NY,New York,Queens,13,1996-04-01,252400.0
3,11235,New York,NY,New York,Kings,14,1996-04-01,190500.0
5,10128,New York,NY,New York,New York,22,1996-04-01,3676700.0


In [108]:
melted_df.tail(20)

,RegionName,City,State,Metro,CountyName,SizeRank,time,value
655060,13061,Nelson,NY,Syracuse,Madison,14562,2018-04-01,153300.0
655061,15015,Bradfordwoods,PA,Pittsburgh,Allegheny,14564,2018-04-01,323300.0
655062,10964,Orangetown,NY,New York,Rockland,14568,2018-04-01,643000.0
655063,14728,Ellery,NY,Jamestown,Chautauqua,14569,2018-04-01,140800.0
655064,14506,Mendon,NY,Rochester,Monroe,14572,2018-04-01,252400.0
655065,12836,Hague,NY,Glens Falls,Warren,14592,2018-04-01,340600.0
655066,10501,Somers,NY,New York,Westchester,14621,2018-04-01,595400.0
655067,16836,Girard,PA,DuBois,Clearfield,14622,2018-04-01,58300.0
655068,8317,Weymouth,NJ,Atlantic City,Atlantic,14633,2018-04-01,165000.0
655069,12824,Bolton,NY,Glens Falls,Warren,14635,2018-04-01,331800.0


In [109]:
df.SizeRank.unique()

array([    7,    11,    13, ..., 14703, 14716, 14718])

In [110]:
melted_df['RegionName'].value_counts()

7676     265
10927    265
11950    265
7848     265
8879     265
        ... 
12458     52
15943     52
12428     52
13797     52
16625     46
Name: RegionName, Length: 2472, dtype: int64

In [111]:
melted_df['value'].describe()

count    6.463560e+05
mean     2.375674e+05
std      3.586677e+05
min      1.300000e+04
25%      9.970000e+04
50%      1.638000e+05
75%      2.878000e+05
max      1.931490e+07
Name: value, dtype: float64

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results